In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

pfr = 'https://www.pro-football-reference.com'

latest_year = 2022
earliest_year = 1970

cols = ['uid','name','pos']
for year in range(latest_year, earliest_year - 1, -1):
    cols.append(str(year))

df = pd.DataFrame(columns=cols)

id_set = set()

for year in range(latest_year, earliest_year - 1, -1):
    r = requests.get(pfr + '/years/' + str(year) + '/fantasy.htm')
    soup = BeautifulSoup(r.content, 'html.parser')
    parsed_table = soup.find_all('table')[0]

    for i,row in enumerate(parsed_table.find_all('tr')[2:]):
        dat = row.find('td', attrs={'data-stat': 'player'})
        try:
            name = dat.a.get_text()
            id = dat.get('data-append-csv')
            pos = row.find('td', attrs={'data-stat': 'fantasy_pos'}).get_text()
            half_ppr = round(float(row.find('td', attrs={'data-stat': 'fantasy_points_ppr'}).get_text()) - (float(row.find('td', attrs={'data-stat': 'rec'}).get_text()) / 2), 1)

            if id not in id_set:
                df.loc[len(df), 'uid'] = id
                df.loc[len(df) - 1, 'name'] = name
                df.loc[len(df) - 1, 'pos'] = pos
                df.loc[len(df) - 1, str(year)] = half_ppr
            else:
                row = df[df['uid']==id].index
                df.loc[row, str(year)] = half_ppr
        except:
            pass
    id_set = set(df['uid'])

In [6]:
df

,id,name,pos,2022,2021,2020,2019,2018,2017,2016,...,1979,1978,1977,1976,1975,1974,1973,1972,1971,1970
0,MahoPa00,Patrick Mahomes,QB,416.9,361.7,374.4,287.0,417.1,10.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JacoJo01,Josh Jacobs,RB,301.8,199.0,214.8,181.6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,McCaCh01,Christian McCaffrey,RB,313.9,109.0,81.9,413.2,332.0,188.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HenrDe00,Derrick Henry,RB,286.3,184.3,323.6,285.6,193.9,129.5,99.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,JeffJu00,Justin Jefferson,WR,304.7,276.4,230.2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5878,RyanFr00,Frank Ryan,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1
5879,DaviDo00,Donnie Davis,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0
5880,ShyxLe00,Les Shy,RB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1
5881,PierDa00,Danny Pierce,RB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.3


In [7]:
df.to_csv('fantasy_points.csv', index=False)

In [8]:
from google.colab import files
files.download('fantasy_points.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>